In [17]:
from pandas import DataFrame
import numpy as np
import os
import re

In [18]:
os.getcwd()

'/root/data/SE-VAE/jupyter'

In [35]:
#origin path list
# Path_list =[
#     'ckpt/winding/vrnn/vrnn_/2020-11-28_13-19-37',
#     'ckpt/winding/vaecl/vaecl_model.D=1/2020-11-29_15-45-38',
#     'ckpt/winding/vaecl/vaecl_model.D=5/2020-11-29_20-42-05',
#     'ckpt/winding/srnn/srnn_/2020-11-25_19-48-03',
#     'ckpt/west/vrnn/vrnn_/2020-11-28_21-59-55',
#     'ckpt/west/vaecl/vaecl_model.D=1/2020-11-29_16-23-01',
#     'ckpt/west/vaecl/vaecl_model.D=5/2020-11-29_22-12-03',
#     'ckpt/west/srnn/srnn_/2020-11-25_21-46-40',
#     'ckpt/cstr/vrnn/vrnn_/2020-11-28_08-51-15',
#     'ckpt/cstr/vaecl/vaecl_model.D=1/2020-11-29_14-11-52',
#     'ckpt/cstr/vaecl/vaecl_model.D=5/2020-11-29_17-21-47',
#     'ckpt/cstr/srnn/srnn_/2020-11-25_14-57-52'
# ]
def my_filter(path):
    log_path = os.path.join('..', path, 'log.out')
    s = open(log_path, 'r').readlines()
    log_path = os.path.join('..', path, 'test.out')
    test_s = open(log_path, 'r').readlines()
    if False:
        return False
    elif 'Error' in test_s[-2]:
        return False
    else:
        return True
    
def generating_dir(base_dir, root_dir):
    ret = []
    for file in os.listdir(os.path.join(root_dir, base_dir)):
        if file == 'tmp':
            continue
        path = os.path.join(base_dir, file)
        if os.path.isdir(os.path.join(root_dir, path)):
            ret = ret + generating_dir(path, root_dir)
        elif file == 'test.out' and os.path.exists(os.path.join(root_dir, base_dir, 'best.pth')):
            print(path)
            if my_filter(os.path.split(path)[0]):
                ret.append(os.path.split(path)[0])
            else:
                continue
    return ret
path_list = generating_dir('ckpt/', '../')


ckpt/winding/odevrnn/ode_vrnn_ct_time=True,model.D=1,model.ode_solver=euler,train.batch_size=512/2022-03-14_17-14-08/test.out
ckpt/winding/odevrnn/ode_vrnn_ct_time=True,model.D=1,model.ode_solver=dopri5,train.batch_size=512_tmp/2022-03-14_20-54-06/test.out
ckpt/winding/odevrnn/ode_vrnn_ct_time=True,model.D=1,model.ode_solver=dopri5,train.batch_size=512/2022-03-14_20-54-06/test.out


In [ ]:
n=len(path_list)
data=[]

data = list(set([path.split('/')[1] for path in path_list]))

#获取数据集/模型名称及个数
# for path in Path_list:
#     #result=re.findall(r'/(.*?)/',path)#查找两个'/'之间的字符
    
#     result=path.split('/')
#     print(result)
#     if result[1] in data:
#         print('repeat data')
#     else:
#         data.append(result[1])

#print(result) 
print(data)

['winding']


In [37]:
#单个frame生成，构建单个数据集path列表
temp_list=[]             #存放单个数据集的临时列表
dex=[]                   #行列表
col=[]                   #列列表
df=[]                    #DataFrame集合
#x=np.zeros((n_dex,n_col))#数值矩阵
for d in range(len(data)):
    #print(d)
    for path in path_list:
        if path.split('/')[1] == data[d]:
            temp_list.append(path)
    #print(temp_list)
    for path in temp_list:
        #pattern=re.compile(r'\/.*?/')
        result=path.split('/')
        #print(result)
        dex.append(result[len(result)-2]+'//'+result[len(result)-1])
    #print('dex', len(dex))
    n_dex=len(dex)#行数
    #print(dex)
    #assert len(dex) == len(temp_list)
    
    #单独开启第一个path文件获取列数以便于初始化数值矩阵
    f = open('../'+temp_list[0]+'/test.out','r')
    temp_data = f.readlines()
    f.close()
    for line in temp_data:
        if re.search('likelihood',line):
            #pattern = re.compile(r'-?[0-9]\d*\.\d*')   #查找数字(之查找包含小数点的)
            t_col = re.findall(r'(\w*\(?\w\)?\w*)=(-?[0-9]\d*\.\d*)',line)
            #print(t_col)
            for i in range(len(t_col)):
                if t_col[i][0]=='time':
                    pass
                else:
                    col.append(t_col[i][0])
    #print('col', len(col))
    n_col=len(col)           #列数
    x=np.zeros((n_dex,n_col))#数值矩阵
    for t in range(len(temp_list)):      #第t个临时列表中的路径
        #print('t=',t)
        f = open('../'+temp_list[t]+'/test.out','r')
        temp_data = f.readlines()
        f.close()
        #数值矩阵生成
        for line in temp_data:
            if re.search('likelihood',line):
                #print(line[11:len(line)])
                #print(len(line))
                pattern = re.compile(r'-?[0-9]\d*\.\d*')   #查找数字(之查找包含小数点的)
                result = pattern.findall(line)
                #数据对应填入该行
                t_x=re.findall(r'(\w*\(?\w\)?\w*)=(-?[0-9]\d*\.\d*)',line)
                #print(t_x, len(t_x))
                #print(t_x)
                for i in range(len(t_x)):
                    if t_x[i][0]=='time':
                        pass
                    else:
                        #print(x.shape, t, i)
                        x[t,i]=t_x[i][1] #由于index填入模型名称的顺序就是index列表的序号，因此行与list一一对应
                #print(x)
    #生成DataFrame
    df.append(DataFrame(x,columns=col,index=dex))
    col=[]   
    dex=[]
    temp_list=[]              #当前数据集frame生成完毕，临时列表清零
    #print('DateFrame accomplished, input df[',d,']to get df_',data[d])
#df


In [38]:
print(data[0])
df[0].to_csv('df_%s.csv' % data[0])
df[0]

winding


,likelihood,ob_rrse,ob_tension1_rrse,ob_tension2_rrse,ob_tension1_pear,ob_tension2_pear,pred_rrse,pred_tension1_rrse,pred_tension2_rrse,pred_tension1_pear,pred_tension2_pear
"ode_vrnn_ct_time=True,model.D=1,model.ode_solver=euler,train.batch_size=512//2022-03-14_17-14-08",331.3102,0.9488,0.7108,1.1869,0.7591,0.3383,1.5581,1.0588,2.0574,0.7434,0.5920
"ode_vrnn_ct_time=True,model.D=1,model.ode_solver=dopri5,train.batch_size=512_tmp//2022-03-14_20-54-06",339.9583,1.0070,0.7034,1.3107,0.7618,0.2179,2.1420,0.8442,3.4397,0.8439,0.3843
"ode_vrnn_ct_time=True,model.D=1,model.ode_solver=dopri5,train.batch_size=512//2022-03-14_20-54-06",339.0423,1.0063,0.6997,1.3130,0.7612,0.2127,2.2359,0.8668,3.6050,0.8374,0.4084


In [23]:
print(data[1])
df[1].to_csv('df_%s.csv' % data[1])
df[1]

IndexError: list index out of range

In [24]:
print(data[2])
df[2].to_csv('df_%s.csv' % data[2])
df[2]

IndexError: list index out of range

In [9]:
print(data[3])
df[3].to_csv('df_%s.csv' % data[3])
df[3]

cstr


,likelihood,ob_rrse,ob_concentration_pear,ob_temp(K)_pear,pred_rrse,pred_concentration_pear,pred_temp(K)_pear
vaecl_model.D=20//2020-12-01_09-51-14,-322.2375,0.1687,0.9802,0.9865,0.2298,0.9731,0.9781
srnn_model.D=1//2020-12-02_13-04-20,-247.0229,0.3162,0.9399,0.9496,0.2278,0.9745,0.9791
srnn_model.D=5//2020-12-02_19-07-53,-273.7450,0.3155,0.9387,0.9523,0.2267,0.9758,0.9813
srnn_model.D=3//2020-12-02_15-44-21,-253.7365,0.3223,0.9341,0.9498,0.2339,0.9747,0.9792
"seq2seq_train.batch_size=128,train.epochs=200,train.max_epochs_stop=30//2021-06-10_04-48-25",0.0052,0.0000,1.0000,1.0000,0.2965,0.9868,0.9891
vaecl_model.D=3//2020-12-01_07-40-36,-403.4965,0.1427,0.9876,0.9909,0.2239,0.9760,0.9792
vaecl_model.D=3//2020-12-01_07-41-34,-418.2319,0.1439,0.9881,0.9905,0.2196,0.9780,0.9802
vaecl_model.D=5//2020-12-01_09-30-04,-353.8404,0.1585,0.9846,0.9885,0.2335,0.9729,0.9777
vaecl_model.D=5//2020-12-01_08-43-09,-374.1149,0.1472,0.9867,0.9895,0.2297,0.9741,0.9779
vaecl_model.D=1//2020-12-01_07-13-54,-413.2803,0.1293,0.9912,0.9922,0.2469,0.9697,0.9731
